In [1]:
import pandas as pd
import panel as pn
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
import psycopg2

In [2]:
db_config = {
    'user': 'postgres',
    'password': '1412',
    'host': 'localhost',
    'port': '5432',
    'database': 'Sistemarm'
}

 
engine = create_engine(f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}")
con = engine.connect()

In [3]:
def read_remedios(id_remedio=None, nome=None):
    try:
        if id_remedio is not None:
            query = text("SELECT * FROM farmacia.remedio WHERE id_remedio = :id_remedio")
            df = pd.read_sql(query, con, params={"id_remedio": id_remedio})
        elif nome is not None:
            query = text("SELECT * FROM farmacia.remedio WHERE nome ILIKE :nome")
            df = pd.read_sql(query, con, params={"nome": f"%{nome}%"})
        else:
            query = "SELECT * FROM farmacia.remedio"
            df = pd.read_sql(query, con)
        return df
    except SQLAlchemyError as e:
        return str(e)

def create_remedio(nome, descricao):
    try:
        query = text("INSERT INTO farmacia.remedio (nome, descricao) VALUES (:nome, :descricao)")
        con.execute(query, {"nome": nome, "descricao": descricao})
        con.commit()   
        return "Remédio adicionado com sucesso!"
    except SQLAlchemyError as e:
        return str(e)

def update_remedio(id_remedio, nome, descricao):
    try:
        query = text("UPDATE farmacia.remedio SET nome = :nome, descricao = :descricao WHERE id_remedio = :id_remedio")
        con.execute(query, {"id_remedio": id_remedio, "nome": nome, "descricao": descricao})
        con.commit()   
        return "Remédio atualizado com sucesso!"
    except SQLAlchemyError as e:
        return str(e)

def delete_remedio(id_remedio):
    try:
        query = text("DELETE FROM farmacia.remedio WHERE id_remedio = :id_remedio")
        con.execute(query, {"id_remedio": id_remedio})
        con.commit()   
        return "Remédio excluído com sucesso!"
    except SQLAlchemyError as e:
        return str(e)


In [4]:
 
pn.extension()

 
def update_table(id_remedio=None, nome=None):
    df = read_remedios(id_remedio, nome)
    table.value = df

 
def on_filter_button_click(event):
    id_filter = id_filter_input.value if id_filter_input.value else None
    nome_filter = nome_filter_input.value if nome_filter_input.value else None
    update_table(id_remedio=id_filter, nome=nome_filter)
    output_pane.object = "Tabela atualizada com base no filtro."

 
id_filter_input = pn.widgets.IntInput(name='Filtrar por ID', width=150)
nome_filter_input = pn.widgets.TextInput(name='Filtrar por Nome', placeholder='Nome do Remédio', width=300)
filter_button = pn.widgets.Button(name='Aplicar Filtro', button_type='primary')

filter_button.on_click(on_filter_button_click)

 
nome_input = pn.widgets.TextInput(name='Nome', placeholder='Nome do Remédio', width=300)
descricao_input = pn.widgets.TextInput(name='Descrição', placeholder='Descrição do Remédio', width=300)
create_button = pn.widgets.Button(name='Adicionar Remédio', button_type='primary')

 
def on_create_button_click(event):
    message = create_remedio(nome_input.value, descricao_input.value)
    update_table()
    output_pane.object = message

create_button.on_click(on_create_button_click)

 
id_input = pn.widgets.IntInput(name='ID do Remédio', width=150)
update_nome_input = pn.widgets.TextInput(name='Novo Nome', placeholder='Novo Nome do Remédio', width=300)
update_descricao_input = pn.widgets.TextInput(name='Nova Descrição', placeholder='Nova Descrição do Remédio', width=300)
update_button = pn.widgets.Button(name='Atualizar Remédio', button_type='warning')

 
def on_update_button_click(event):
    message = update_remedio(id_input.value, update_nome_input.value, update_descricao_input.value)
    update_table()
    output_pane.object = message

update_button.on_click(on_update_button_click)

 
delete_id_input = pn.widgets.IntInput(name='ID do Remédio para Excluir', width=150)
delete_button = pn.widgets.Button(name='Excluir Remédio', button_type='danger')

 
def on_delete_button_click(event):
    message = delete_remedio(delete_id_input.value)
    update_table()
    output_pane.object = message

delete_button.on_click(on_delete_button_click)

 
table = pn.widgets.DataFrame(read_remedios(), name='Tabela de Remédios', width=600)
output_pane = pn.pane.Markdown("Mensagens aparecerão aqui", width=600)

 
layout = pn.Column(
    pn.pane.Markdown("# Gerenciamento de Remédios", sizing_mode='stretch_width'),
    pn.Row(
        pn.Column(id_filter_input, nome_filter_input, filter_button),
        sizing_mode='stretch_width'
    ),
    pn.Row(
        pn.Column(nome_input, descricao_input, create_button),
        sizing_mode='stretch_width'
    ),
    pn.Row(
        pn.Column(id_input, update_nome_input, update_descricao_input, update_button),
        sizing_mode='stretch_width'
    ),
    pn.Row(
        pn.Column(delete_id_input, delete_button),
        sizing_mode='stretch_width'
    ),
    table,  
    output_pane,
    sizing_mode='stretch_width'
)
layout.show()


C:\Users\taila\AppData\Local\Temp\ipykernel_16508\3231726030.py:1: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension()


Launching server at http://localhost:58769
